In [1]:
import sys 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils
import math
from sklearn.cluster import KMeans
%matplotlib

Using matplotlib backend: Qt5Agg


In [31]:
def task1(image1):
    image = cv2.imread(image1)
    image = imutils.resize(image, width=600, height=480)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image,None)
    img = cv2.drawKeypoints(gray,keypoints,image,color=(0,255,0),flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return img, keypoints, rgb

In [32]:
def drawKeyPts(im,keyp,th):
    for curKey in keyp:
        x=np.int(curKey.pt[0])
        y=np.int(curKey.pt[1])
        size = np.int(curKey.size)
        cv2.drawMarker(im, (x, y), (255,0,0), markerSize = th,);  
    return im    

In [33]:
def draw(imgKey, rgb):
    dpi = 80
    height, width, depth = imgKey.shape
    figsize = (width / float(dpi)*2), (height / float(dpi)*2)
    figure = plt.subplots(figsize=figsize)
    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.title("Original Image(rescaled)")
    plt.imshow(rgb)
    plt.subplot(1, 2, 2)
    plt.imshow(imgKey)
    plt.axis('off')
    plt.title("Image with highlighted keypoints")
    plt.subplots_adjust(wspace=0.025, hspace=0.05)
    plt.show()

In [49]:
name = "img08.JPG"
img, keypoints, rgb = task1(name)
imWithCross = drawKeyPts(img,keypoints,7)
print("# of keypoints in " + name + " is", len(keypoints))
draw(imWithCross, rgb)

# of keypoints in img08.JPG is 4153


TASK 2

In [47]:
def task2(img_array):
    for i in range(len(img_array)):
        img_array[i] = imutils.resize(img_array[i], width=600, height=480)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints_arr = []
    descriptors_arr = [] 
    features = []
    for i in img_array:
        keypoints, descriptors = sift.detectAndCompute(i,None)
        keypoints_arr.append(keypoints)
        descriptors_arr.extend(descriptors)
        features.append(descriptors)
    total_keypoints = 0
    for i in range(len(keypoints_arr)):
        total_keypoints += len(keypoints_arr[i])
        print("# of keypoints in image {} is {}".format(i+1, len(keypoints_arr[i])))
    print("Total # of keypoints are", total_keypoints)
    print("............................")
    k_count = 0
    k_arr = [math.floor(0.05*total_keypoints), math.floor(0.1*total_keypoints), math.floor(0.2*total_keypoints)]
    for k in k_arr:
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(descriptors_arr)
        histograms = []
        for i in img_array:
            histo = np.zeros(k)
            kp, des = sift.detectAndCompute(i, None)
            nkp = np.size(kp)
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp
            histograms.append(histo)
        for i in range(len(histograms)):
            histograms[i] = histograms[i].astype('float32') 
        distances = []
        for i in range(len(histograms)):
            dis = []
            for j in range(len(histograms)):
                dis.append(cv2.compareHist(histograms[i], histograms[j], cv2.HISTCMP_CHISQR))
            distances.append(dis)
        for i in range(len(histograms)):
            for j in range(len(histograms)):
                distances[i][j] = round(distances[i][j], 2)
        ny = np.array(distances)
        title = ["A", "B", "C"]
#         for i in range(1, len(sys.argv)+1):
#             title.append(sys.argv[i])
        k_values = [5, 10, 20]
        print("K = {}%*(total number of keypoints) = {}".format(k_values[k_count], k))
        k_count += 1
        print("Dissimilarity matrix (Upper triangular matrix)")
        print("\t", end = "")
        for i in title:
            print(i, end = "\t")
        print("\n")
        for i in range(len(histograms)):
            print(title[i], end = "\t")
            for j in range(len(histograms)):
                print(np.triu(ny,k=0)[i][j], end = "\t")
            print("\n")
        print("\n")

In [48]:
count = len(sys.argv)-1
img_array = []
for i in range(1, len(sys.argv)+1):
    img_array.append(cv2.imread(sys.argv[i]))

imgs = ["img14.JPG", "img15.JPG"]
img_array = []
for i in range(len(imgs)):
    img_array.append(cv2.imread(imgs[i]))
task2(img_array)

# of keypoints in image 1 is 761
# of keypoints in image 2 is 1260
Total # of keypoints are 2021
............................
K = 5%*(total number of keypoints) = 101
Dissimilarity matrix (Upper triangular matrix)
	A	B	C	

A	0.0	2.04	

B	0.0	0.0	



K = 10%*(total number of keypoints) = 202
Dissimilarity matrix (Upper triangular matrix)
	A	B	C	

A	0.0	1.39	

B	0.0	0.0	



K = 20%*(total number of keypoints) = 404
Dissimilarity matrix (Upper triangular matrix)
	A	B	C	

A	0.0	0.85	

B	0.0	0.0	



